In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import requests
import csv
import re
import mysql.connector 

# Configuración
CLIENT_ID = '9189e74191a848bf82c60efbf06229c6'
CLIENT_SECRET = '36c5b1edfa3744e7ac992361fa2b5b02'
API_KEY_LASTFM = '472626d71b58f9fbdd11821189cb8716'
BASE_URL = "http://ws.audioscrobbler.com/2.0/"

# Configuración MySQL
DB_CONFIG = {
    'user': 'root',
    'password': 'AlumnaAdalab',
    'host': '127.0.0.1',
    'database': 'proyecto_grupal_mod_2'
}

# Función 1: Conectar con Spotify
def conectar_spotify():
    
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET
    ))
    return sp

# Función 2: Buscar canciones de un género y año
def buscar_canciones(sp, genero, ano):
    
    canciones = []
    
    # Buscar en dos páginas (0 y 50)
    for pagina in [0, 50]:
        resultados = sp.search(q=f'genre:"{genero}" year:{ano}', 
                              type='track', 
                              limit=50, 
                              offset=pagina)
        
        for cancion in resultados['tracks']['items']:
            # Obtener nombres de artistas
            artistas = []
            for artista in cancion['artists']:
                artistas.append(artista['name'])
            
            # Crear diccionario con datos de la canción
            datos_cancion = {
                'artistas': ', '.join(artistas),
                'colaboracion': len(artistas) > 1,
                'genero_musical': genero,
                'nombre': cancion['name'],
                'ano_lanzamiento': cancion['album']['release_date'][:4],
                'id': cancion['id']
            }
            canciones.append(datos_cancion)
    
    return canciones

# Función 3: Obtener todas las canciones
def obtener_todas_canciones():
    
    # Conectar con Spotify
    sp = conectar_spotify()
    
    # Listas de géneros y años
    generos = ['pop', 'metal', 'reggaeton', 'rap']
    años = [2015, 2016]
    
    # Lista para guardar todas las canciones
    todas_canciones = []
    
    # Buscar canciones para cada género y año
    for genero in generos:
        print(f"Buscando canciones de {genero}...")
        for ano in años:
            print(f"  Año {ano}...")
            canciones = buscar_canciones(sp, genero, ano)
            todas_canciones.extend(canciones)
    
    return todas_canciones

# Función 4: Guardar canciones en CSV
def guardar_canciones_csv(canciones):
    
    with open('spotify_datos.csv', 'w', newline='', encoding='utf-8') as archivo:
        campos = ['artistas', 'colaboracion', 'genero_musical', 'nombre', 'ano_lanzamiento', 'id']
        escritor = csv.DictWriter(archivo, fieldnames=campos)
        escritor.writeheader()
        escritor.writerows(canciones)
    
    print(f"✅ Guardado: spotify_datos.csv ({len(canciones)} canciones)")

# Función 5: Leer artistas del CSV
def leer_artistas_csv():
    
    df = pd.read_csv('spotify_datos.csv')
    
    # Lista para todos los artistas
    todos_artistas = []
    
    # Procesar cada fila
    for fila in df['artistas']:
        # Separar artistas por comas
        lista_artistas = fila.split(',')
        for artista in lista_artistas:
            artista_limpio = artista.strip()  # Quitar espacios
            if artista_limpio:  # Si no está vacío
                todos_artistas.append(artista_limpio)
    
    # Quitar duplicados y ordenar
    artistas_unicos = list(set(todos_artistas))
    artistas_unicos.sort()
    
    print(f"Artistas únicos encontrados: {len(artistas_unicos)}")
    return artistas_unicos

# Función 6: Quitar etiquetas HTML
def quitar_html(texto):
    
    if texto:
        texto_limpio = re.sub('<[^<]+?>', '', texto)
        return texto_limpio.strip()
    return ''

# Función 7: Buscar información de un artista en Last.fm
def buscar_artista_lastfm(nombre_artista):
    
    # Parámetros para la API
    parametros = {
        "method": "artist.getInfo",
        "artist": nombre_artista,
        "api_key": API_KEY_LASTFM,
        "format": "json",
        "lang": "es"
    }
    
    # Hacer petición a la API
    respuesta = requests.get(BASE_URL, params=parametros)
    datos = respuesta.json()
    
    # Verificar si encontró el artista
    if "artist" in datos:
        artista_info = datos['artist']
        
        # Obtener biografía
        biografia = ''
        if 'bio' in artista_info and 'content' in artista_info['bio']:
            biografia = quitar_html(artista_info['bio']['content'])
        
        # Obtener estadísticas
        playcount = 0
        oyentes = 0
        if 'stats' in artista_info:
            playcount = artista_info['stats'].get('playcount', 0)
            oyentes = artista_info['stats'].get('listeners', 0)
        
        # Obtener artistas similares
        similares = []
        if 'similar' in artista_info and 'artist' in artista_info['similar']:
            for similar in artista_info['similar']['artist']:
                similares.append(similar.get('name', ''))
        
        # Crear diccionario con la información
        info_artista = {
            'artist': nombre_artista,
            'biografia': biografia,
            'playcount': playcount,
            'oyentes': oyentes,
            'similares': ', '.join(similares)
        }
        
        return info_artista
    
    return None

# Función 8: Obtener información de todos los artistas
def obtener_info_artistas():
    
    # Leer artistas del CSV
    artistas = leer_artistas_csv()
    
    # Lista para guardar información de artistas
    info_artistas = []
    
    # Buscar información de cada artista
    contador = 0
    for artista in artistas:
        contador += 1
        print(f"Procesando artista {contador}/{len(artistas)}: {artista}")
        
        info = buscar_artista_lastfm(artista)
        if info:
            info_artistas.append(info)
    
    return info_artistas

# Función 9: Guardar información de artistas en CSV
def guardar_artistas_csv(info_artistas):
    
    with open('lastfm_datos.csv', 'w', newline='', encoding='utf-8') as archivo:
        campos = ['artist', 'biografia', 'playcount', 'oyentes', 'similares']
        escritor = csv.DictWriter(archivo, fieldnames=campos)
        escritor.writeheader()
        escritor.writerows(info_artistas)
    
    print(f"✅ Guardado: lastfm_datos.csv ({len(info_artistas)} artistas)")

# Función 10: Proceso completo de Spotify
def proceso_spotify():
    
    print("🎵 Iniciando proceso de Spotify...")
    
    # Obtener canciones
    canciones = obtener_todas_canciones()
    
    # Guardar en CSV
    guardar_canciones_csv(canciones)
    
    print("✅ Proceso de Spotify completado")

# Función 11: Proceso completo de Last.fm
def proceso_lastfm():
    
    print("🎤 Iniciando proceso de Last.fm...")
    
    # Obtener información de artistas
    info_artistas = obtener_info_artistas()
    
    # Guardar en CSV
    guardar_artistas_csv(info_artistas)
    
    print("✅ Proceso de Last.fm completado")

# Función 12: Proceso completo
def proceso_completo():
    
    print("🚀 Iniciando proceso completo...")
    
    # Primero Spotify
    proceso_spotify()
    
    # Luego Last.fm
    proceso_lastfm()
    
    print("✅ ¡Todo terminado!")

# Función 13: Conectar con MySQL
def conectar_mysql():
    
    print("🔗 Conectando con MySQL...")
    
    cnx = mysql.connector.connect(**DB_CONFIG)
    print("✅ Conexión establecida:", cnx)
    
    # Verificar base de datos activa
    mycursor = cnx.cursor()
    mycursor.execute("SELECT DATABASE();")
    db_actual = mycursor.fetchone()
    print("Base de datos activa:", db_actual)
    mycursor.close()
    
    return cnx

# Función 14: Insertar artistas en MySQL
def insertar_artistas_mysql():
    
    print("🎤 Insertando artistas en MySQL...")
    
    # Conectar con MySQL
    cnx = conectar_mysql()
    mycursor = cnx.cursor()
    
    # Leer archivo CSV
    with open('lastfm_datos.csv', 'r', encoding='utf-8') as archivo_csv:
        reader = csv.DictReader(archivo_csv)
        
        for fila in reader:
            sql = """
                INSERT INTO artistas (nombre, biografia, playcount, oyentes, similares)
                VALUES (%s, %s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE 
                    biografia = VALUES(biografia),
                    playcount = VALUES(playcount),
                    oyentes = VALUES(oyentes),
                    similares = VALUES(similares)
            """
            
            valores = (
                fila['artist'],
                fila['biografia'],
                int(fila['playcount']),
                int(fila['oyentes']),
                fila.get('similares', '')
            )
            
            mycursor.execute(sql, valores)
    
    # Guardar cambios
    cnx.commit()
    mycursor.close()
    cnx.close()
    
    print("✅ Artistas insertados en MySQL correctamente")

# Función 15: Insertar canciones en MySQL
def insertar_canciones_mysql():
    
    print("🎵 Insertando canciones en MySQL...")
    
    # Conectar con MySQL
    cnx = conectar_mysql()
    mycursor = cnx.cursor()
    mycursor_bucle = cnx.cursor()
    
    # Leer archivo CSV
    with open('spotify_datos.csv', 'r', encoding='utf-8') as archivo_csv:
        reader = csv.DictReader(archivo_csv)
        
        for fila in reader:
            # 1. Insertar canción
            sql_cancion = """
                INSERT INTO canciones (nombre, genero_musical, tipo, año_lanzamiento, id_spotify)
                VALUES (%s, %s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE nombre = VALUES(nombre)
            """
            
            valores_cancion = (
                fila['nombre'],
                fila['genero_musical'],
                fila['genero_musical'],
                int(fila['ano_lanzamiento']),
                fila['id']
            )
            
            mycursor.execute(sql_cancion, valores_cancion)
            
            # 2. Obtener ID de la canción
            mycursor_bucle.execute("SELECT id_cancion FROM canciones WHERE id_spotify = %s", (fila['id'],))
            id_cancion = mycursor_bucle.fetchone()[0]
            
            # 3. Insertar relación canción-artista
            artistas = [art.strip() for art in fila['artistas'].split(',')]
            for artista in artistas:
                # Buscar ID del artista
                mycursor_bucle.execute("SELECT id_artista FROM artistas WHERE nombre = %s", (artista,))
                res = mycursor_bucle.fetchone()
                
                if res:
                    id_artista = res[0]
                    sql_relacion = """
                        INSERT INTO canciones_artistas (id_cancion, id_artista)
                        VALUES (%s, %s)
                        ON DUPLICATE KEY UPDATE id_cancion = id_cancion
                    """
                    mycursor_bucle.execute(sql_relacion, (id_cancion, id_artista))
    
    # Guardar cambios
    cnx.commit()
    mycursor.close()
    mycursor_bucle.close()
    cnx.close()
    
    print("✅ Canciones insertadas en MySQL correctamente")

# Función 16: Insertar todos los datos en MySQL
def insertar_datos_mysql():
    
    print("💾 Insertando todos los datos en MySQL...")
    
    # Primero insertar artistas
    insertar_artistas_mysql()
    
    # Luego insertar canciones
    insertar_canciones_mysql()
    
    print("✅ Todos los datos insertados en MySQL")

# Función 17: Proceso completo con MySQL
def proceso_completo_mysql():
    
    print("🚀 Iniciando proceso completo con MySQL...")
    
    # 1. Proceso de Spotify
    proceso_spotify()
    
    # 2. Proceso de Last.fm
    proceso_lastfm()
    
    # 3. Insertar en MySQL
    insertar_datos_mysql()
    
    print("✅ ¡Todo terminado con MySQL!")

# Función 18: Solo MySQL (solo inserta los datos si ya tienes los CSV)
def solo_mysql():
    
    print("💾 Insertando datos existentes en MySQL...")
    insertar_datos_mysql()

# Ejecutar el programa
if __name__ == "__main__":
    # Opción 1: Proceso completo con MySQL
    proceso_completo_mysql()
    
    # Opción 2: Solo MySQL (descomenta la línea siguiente)
    

🚀 Iniciando proceso completo con MySQL...
🎵 Iniciando proceso de Spotify...
Buscando canciones de pop...
  Año 2015...
  Año 2016...
Buscando canciones de metal...
  Año 2015...
  Año 2016...
Buscando canciones de reggaeton...
  Año 2015...
  Año 2016...
Buscando canciones de rap...
  Año 2015...
  Año 2016...
✅ Guardado: spotify_datos.csv (800 canciones)
✅ Proceso de Spotify completado
🎤 Iniciando proceso de Last.fm...
Artistas únicos encontrados: 833
Procesando artista 1/833: 13 Steps
Procesando artista 2/833: 38 Spesh
Procesando artista 3/833: 46Simmy
Procesando artista 4/833: 679
Procesando artista 5/833: 8Ball & MJG
Procesando artista 6/833: 995
Procesando artista 7/833: A Constant Storm
Procesando artista 8/833: A$AP ANT
Procesando artista 9/833: AFROJACK
Procesando artista 10/833: ARY
Procesando artista 11/833: Abandoned Elysium
Procesando artista 12/833: Abida Parveen
Procesando artista 13/833: Abigorum
Procesando artista 14/833: Absent Minded
Procesando artista 15/833: Aeternu